In [1]:
%load_ext Cython

load combine record

In [2]:
# %load combine_record.py
"""
Spyder Editor

This is a combined wechat record analysis script file.

run order 0.0
output: a combined wechat group chattting log

"""
#%%
import numpy as np
import pandas as pd
import os
import re
import chardet
import datetime
os.chdir(r'D:\\0Knowledge\\Fudan\\0.20170412kidswant\\wechat\\py_wechat')
#%%

#%%
"""
how to get wechat records:
https://www.zhihu.com/question/19924224

tables:
message 
  send_msg凡是含有appid的，到appinfo表中进行匹配，例如appid=\"\"wx992c420c8d64dc18\"\"
          凡是含有title的，<title>少儿手足口病险（秒杀专享）</title>，也取出来
  type :1常规文本3应用消息49其他应用分享的信息，可以通过<title>获取47表情10000系统提示信息，加入群聊，撤回消息等
    34图片 43图片436207665红包 268435505 应用信息 42 估计是名片 48位置共享 1048625不明，去匹配Appmessage通过msgid
    452984881 去匹配appmessage 16777265匹配appmessage 64语音聊天的系统提示 10000进群退群提示 -1879048186去匹配
    
    基本上就是除了1以外，其他的从appmessage进行匹配
rcontact 联系人，个人的表
chatroom0830 群聊的基本信息
AppMessage 各种应用的提示信息，包括微信自己的红包和第三方的应用分享
EmojiInforDesc 表情描述
appinfor0830涉及到的应用的描述

大概就是这些会比较有用
"""

'''
首先是将微信的聊天记录合并起来
并对数据做一些初步的处理
'''
#%%
#%%
'''
#get group display name from the html source code
#parse the code with re and extract the displayname
#这里得到的是全部的群昵称
'''
#match chatroom name in Chinese with chatroom id .
match_chatroom = pd.read_csv(r'../records/match_chatroom.csv', encoding = 'gb2312')
basepath = u'../records/nicknames/'
'''
input : the directory of html source codes
output : a df with group display names
'''
def parseDisplayname(dir):
    displaydf = pd.DataFrame(columns = ['roomname','displayname'])    
    for file in os.listdir(dir):
        chatroomname = file
        with open(basepath + file,'r', encoding = 'utf-8') as f:
            htmlfile = f.read()
            
            #re.complie 将用到的正则表达式预先编码，提高速度，模式re.S，实现跨行匹配
            #注意正则表达式里里面的.和()都需要进行转义
            re_dis = re.compile(r'getDisplayName\(currentContact\.UserName\)">(.*?)</p>',re.S)
            nicknames = re_dis.findall(htmlfile)
            
            tmpdf = pd.DataFrame({
                    'roomname':chatroomname,
                    'displayname':nicknames
                    })
            displaydf = displaydf.append(tmpdf,ignore_index = True)
    return(displaydf)
#这里得到的昵称是群昵称，也就是聊天的时候显示的名字
#而且，这里的个数只会比chatroom的行数多吧。。。
#这里假设没有人退群
displaynames = parseDisplayname(basepath)#16356
#先假设没有人退群，做merge处理，比对通过roomdata得到的displayname
#drop 掉两个群，武进万达孕妈群和辣妈帮
displaynames.drop_duplicates(subset = ['displayname','roomid'], inplace = True)
displaynames = displaynames.merge(match_chatroom, left_on = 'roomname', right_on = 'name', how = 'inner')#16356

#%%
#%%
'''
#read
#第一个备份从5月到8月
'''
message0830 = pd.read_csv(r'../records/170830/message.csv')
chatroom0830 = pd.read_csv(r'../records/170830/chatroom.csv')
appmsg0830 = pd.read_csv(r'../records/170830/AppMessage.csv')
appinfor0830 = pd.read_csv(r'../records/170830/appinfor.csv')
#%%

#filter chatroom0830
#%%
nokidswantgroup = ["7262310752@chatroom","6510569027@chatroom"]#非孩子王的群
chatroom0830 = chatroom0830[~chatroom0830.chatroomname.isin(nokidswantgroup)]
remove_room = message0830.talker[message0830.content.str.contains(r'"移出群聊').fillna(False)]#被5个群移除群聊
chatroom0830 = chatroom0830[~chatroom0830.chatroomname.isin(remove_room)]#现在有79个群
#%%

#%%
def match_nickname(chatroomdf):
    '''
    to get nicknames of members in a group, but the question is when people chat in group, the name displayed in 
    group chatting is not always the nickname but sometimes the group displayname if they set this name, which can split from the roomdata column.
    But! this function is still usable for those chatroom without fault.
    
    In conclusion, this function get nicknames in the right chatroom
    '''
    matched_df = pd.DataFrame(columns = ['chatroomname','member','nickname'])
    for row in chatroomdf.itertuples():
        chatroomname = row[1]
        memberset = row[3]
        displaynameset = row[4]
        memberlist = memberset.split(';')
        displaynamelist = displaynameset.split('、')
        if len(memberlist) == len(displaynamelist):
            for i in range(len(memberlist)):
                tmpdf = pd.DataFrame({
                        'chatroomname':chatroomname,
                        'member':memberlist[i],
                        'nickname':displaynamelist[i]                    
                        },index = [0])
                matched_df = matched_df.append(tmpdf,ignore_index=True)
    return(matched_df)
    
def match_groupdisplayname(chatroomdf,nicknamedf):
    '''
    to match the member wechat id and group display name in roomdata, which can 
    not be derived from other columns in chatroom dataframe
    '''
    matched_df = pd.DataFrame(columns = ['chatroomname','member','displayname'])
    for row in chatroomdf.itertuples():
        chatroomname = row[1]        
        totalset = row[8]
        #splitby \n,but not DC2\n
        tmp1 = re.split('(?<!\x12)\n',totalset)
        #split by ASCII control codes
        tmp2 = [re.split('[\x00-\x1F]',x) for x in tmp1]
        wechatid = []
        displayname = []
        tmp3 = []
        for i in tmp2:
            if len(i) <2:
                tmp2.remove(i)
            else:
                while '' in i:
                    i.remove('')
                tmp3.append(i)
                    
                    
        for i in tmp3:
            if len(i) > 1:
                wechatid.append(i[0])
                displayname.append(i[1])
        
        matched_df = matched_df.append(pd.DataFrame({
                'chatroomname':chatroomname,
                'member':wechatid,
                'displayname':displayname
                }),ignore_index = True)
    #将得到的群昵称和微信昵称进行外链接
    matched_df = matched_df.merge(nicknamedf,on = ['chatroomname','member'],how = 'outer')
    return(matched_df)
        

nickname0830 = match_nickname(chatroom0830)
nickname0830 = match_groupdisplayname(chatroom0830,nickname0830)
#%%

#%%
'''
#get sender and msg
#在content中提取sender和发送的内容
'''
def get_sender_msg(msgdf):
    msgdf['sender'] = msgdf.content.str.split(':\n').str[0]
    msgdf['sender'][msgdf.type == 10000] = np.NaN
    msgdf['msg'] = msgdf.content.str.split(':\n',n=1).str[1]
#删除msgId列，因为没用,inplace的意思是是否在原始数据上进行操作
#删除content列，因为和sender，msg重复了
    msgdf.drop(columns = ['msgId','content'], inplace = True)
#%%


#%%
'''
 #msg filter modigy and combine
'''
message0830.createTime = pd.to_datetime(message0830.createTime,unit = 'ms')
message0830 = message0830[message0830.talker.isin(chatroom0830.chatroomname)]


get_sender_msg(message0830)
message0830 = message0830.merge(nickname0830,left_on = ['talker','sender'],right_on = ['chatroomname','member'],how = 'left')
#所以说total_msg中是将displayname和nickname整合了的
total_msg = message0830[['msgSvrId','type','status','isSend','createTime','talker','sender','displayname','nickname','msg']]
#%%


#%%
'''
#另一个手机上的备份，从10月到现在
#以后的聊天记录备份都可以在该部分处理
'''
message180119 = pd.read_csv(r'../records/180119/message.csv')
chatroom180119 = pd.read_csv(r'../records/180119/chatroom.csv')
appmsg180119 = pd.read_csv(r'../records/180119/appmessage.csv')
appinfor180119 = pd.read_csv(r'../records/180119/appinfor.csv')
#180213是180119的增量更新，可以直接覆盖
message180213 = pd.read_csv(r'../records/180213/message.csv')
chatroom180213 = pd.read_csv(r'../records/180213/chatroom.csv')
appmsg180213 = pd.read_csv(r'../records/180213/appmessage.csv')
appinfor180213 = pd.read_csv(r'../records/180213/appinfo.csv')
#180310不是上一次的增量更新，需要和之前的合并
message180310 = pd.read_csv(r'../records/180310/message.csv')
chatroom180310 = pd.read_csv(r'../records/180310/chatroom.csv')
appmsg180310 = pd.read_csv(r'../records/180310/appmessage.csv')
appinfor180310 = pd.read_csv(r'../records/180310/appinfor.csv')

#%%

#%%
'''
chatroom filter 
'''
chatroom180119 = chatroom180119[~chatroom180119.chatroomname.isin(nokidswantgroup)]
message180119.talker[message180119.content.str.contains('"移出群聊').fillna(False)]
remove_room = pd.concat([remove_room,message180119.talker[message180119.content.str.contains('"移出群聊').fillna(False)]],ignore_index = True).drop_duplicates().reset_index(drop = True)

chatroom180310 = chatroom180310[~chatroom180310.chatroomname.isin(nokidswantgroup)]
message180310.talker[message180310.content.str.contains('"移出群聊').fillna(False)]#2月份被6个群移除了群聊
#和之前的remove_room合并
remove_room = pd.concat([remove_room,message180310.talker[message180310.content.str.contains('"移出群聊').fillna(False)]],ignore_index = True).drop_duplicates().reset_index(drop = True)
#这里的remove_room是第一个的子集
#所以直接使用
chatroom180119 = chatroom180119[~chatroom180119.chatroomname.isin(remove_room)]
chatroom180310 = chatroom180310[~chatroom180310.chatroomname.isin(remove_room)]
#%%

#%%
'''
get the nickname and group display name
'''
nickname180119 = match_nickname(chatroom180119)
nickname180119 = match_groupdisplayname(chatroom180119,nickname180119)
nickname180213 = nickname180119#2月份的记录使用的1月份的chatroom

nickname180310 = match_nickname(chatroom180310)
nickname180310 = match_groupdisplayname(chatroom180310, nickname180310)
#%%


#%%
'''
message filter modify and combine
'''
message180213 = message180213[message180213.talker.isin(chatroom180119.chatroomname)]#其实此刻有两个群被移除，但是移除的时间并不长，所以不做处理
get_sender_msg(message180213)
#将display name merge到message上
message180213 = message180213.merge(nickname180213,left_on = ['talker','sender'],right_on = ['chatroomname','member'],how = 'left')
message180213.createTime = pd.to_datetime(message180213.createTime,unit='ms')
total_msg = pd.concat([total_msg,message180213],join='inner',ignore_index = True)

message180310 = message180310[message180310.talker.isin(chatroom180310.chatroomname)]
get_sender_msg(message180310)
#将display name merge到message上
message180310 = message180310.merge(nickname180310,left_on = ['talker','sender'],right_on = ['chatroomname','member'],how = 'left')
message180310.createTime = pd.to_datetime(message180310.createTime, unit = 'ms')
total_msg = pd.concat([total_msg,message180310],join='inner',ignore_index = True)
#%%


#按照msgSvrId进行去重
#这个操作一定要在从模拟器导入数据之前，因为这些数据没有SvrId
total_msg.drop_duplicates('msgSvrId',inplace = True)

#%%
'''
#解决聊天中的@问题
#这个方法不适合通过软件导出的记录
'''
#displaynames匹配wechat id
tmpdf = pd.DataFrame()
for room in displaynames.roomid.unique():
    members = chatroom180310[chatroom180310.chatroomname == room].memberlist.tolist()[0].split(';')
    members = pd.Series(members)
    '''
    join : {‘inner’, ‘outer’}, default ‘outer’. How to handle indexes on other axis(es). Outer for union and inner for intersection.
ignore_index : boolean, default False. If True, do not use the index values on the concatenation axis. The resulting axis will be labeled 0, ..., n - 1. This is useful if you are concatenating objects where the concatenation axis does not have meaningful indexing information.
!!! Note the index values on the other axes are still respected in the join.
So what i need to do is reindexing the sub displaynams dataframe to 0.1.2... and set join to inner, cause now it is out join and that's awarked
    '''
    tmp = pd.concat([displaynames[displaynames.roomid == room].reset_index(drop = True),members],axis = 1, ignore_index = True)
    tmpdf = pd.concat([tmpdf,tmp],ignore_index = True)
    #end of for
displaynames = tmpdf.dropna()
displaynames.columns = ['displayname','roomname','name','roomid','member']

#@人名通过unicode中的\u2005提取
re_unicode = re.compile(u'@(?P<atname>.*?)\u2005')

#msgat = total_msg.msg.str.decode('utf-8').str.extractall(re_unicode)#102397 py2
msgat = total_msg.msg.str.extractall(re_unicode)#102397 在py3中文本总是unicode不用担心编码问题，只需要在读取的时候指明编码方式即可
msgat.index.levels[0].name = 'msgindex'
total_msg.index.name = 'msgindex'
# join with index, but the index should be named first
msgat = msgat.join(total_msg[['msgSvrId','createTime','talker','sender']])#102397

#msgat.atname = msgat.atname.str.encode('utf-8') py3中不再需要
msgat = msgat.merge(displaynames,left_on = ['atname','talker'],right_on = ['displayname','roomid'],how = 'left')#102397
#只需要member和displayname的对照
msgat = msgat[['msgSvrId','atname','createTime','talker','member','sender']]
msgat.columns = ['msgSvrId','atname','createTime','talker','member_x','sender']
'''
在和displaynames表匹配完后，和nicknames表的匹配可以用如下的函数进行
分别匹配displaynam和nickname
'''
def concatNames(atdf,nicknamedf):
    #match the displayname
    atdf = atdf.merge(nicknamedf,left_on = ['atname','talker'],right_on = ['displayname','chatroomname'],how = 'left')
    atdf.member_x[atdf.member_x.isna()] = atdf.member[atdf.member_x.isna()]
    atdf = atdf[['msgSvrId','atname','createTime','talker','member_x','sender']]
    #match the nickname
    atdf = atdf.merge(nicknamedf,left_on = ['atname','talker'],right_on = ['nickname','chatroomname'],how = 'left')
    atdf.member_x[atdf.member_x.isna()] = atdf.member[atdf.member_x.isna()]
    atdf = atdf[['msgSvrId','atname','createTime','talker','member_x','sender']]
    return atdf
    
msgat = concatNames(msgat,nickname0830)
msgat = concatNames(msgat,nickname180119)
msgat = concatNames(msgat,nickname180310)

len(msgat[msgat.member_x.isna()])*1.0/len(msgat)#有53%的找不到@对应的人
#如果找不到displayname对应的wechatid，则删掉
msgat.dropna(inplace = True, subset = ['member_x'])
nickname0830.dropna(subset = ['displayname']).pipe(lambda x: x.loc[x.displayname.str.contains('赛')])
nickname0830.dropna(subset = ['nickname']).pipe(lambda x: x.loc[x.nickname.str.contains('赛')])

'''
#ToDO
用聊天日志中的@名称去匹配displaynames，发现有很多找不到（6701个昵称找不到），试图通过昵称和房间号去nicknames的四个表中去寻找；
但是，通过模拟器取得的那些聊天不需要通过这种方法，因为本身记录的就是群昵称，先通过这部分进行算法的训练
但是，软件导出的聊天记录也不是全部都有displayname，还是残缺的
'''
def find_displaynames(df):
    #for row in df.itertuples():
        #if(row.createTime )
    return 0

#2017-12-07 04:00:48.001000

#%%

#从模拟器中导出的聊天记录
#from '2017-08-30 12:27:50' to '2017-10-21 19:17:49'
#导出的文件含有乱码，这个问题如何解决
#%%
msg_from_simul = pd.read_csv(r'../records/171021/total_msg(2).csv')
msg_from_simul.columns = ['talker','msg','createTime','sender']

msg_from_simul['displayname'] = msg_from_simul.sender.str.extract('^(.*?)\(', expand = False)
msg_from_simul.sender = msg_from_simul.sender.str.extract('.*\\((.*)\\).*', expand = False)
msg_from_simul.createTime = pd.to_datetime(msg_from_simul.createTime)
#这部分的中文displayname含有乱码，先不使用
#total_msg = pd.concat([total_msg,msg_from_simul],join='outer',ignore_index = True)
#%%

#数据整合工作完成！！！


#%%
#观察每个月的活跃度
#total_msg.groupby(total_msg.createTime.dt.month+total_msg.createTime.dt.year*100).size()
#%%

#%%
#将2018年2月份（含）的聊天记录输出到csv
#这里需要注意，最后需要的可能只是所有完整的log，也就是说要使用最新的chatroom进行筛选
total_msg[(total_msg.createTime < '2018-03-01')&(total_msg.talker.isin(chatroom180310.chatroomname))].to_pickle(r'../records/sample/sample_msg_pickle')
#%%




D:\Anaconda2\envs\py3\lib\site-packages\ipykernel_launcher.py:186: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda2\envs\py3\lib\site-packages\ipykernel_launcher.py:330: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda2\envs\py3\lib\site-packages\ipykernel_launcher.py:334: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


load classifiertrain.py

In [ ]:
# %load classifiertrain.py
"""
Created on Mon Mar 12 21:47:31 2018

@author: Eric

run order:1

"""
#%%
import scipy.stats as st
import matplotlib.pyplot as plt
import nltk
import math
import timeit
#nltk.download()
from pyltp import SentenceSplitter
#%%

#%%
'''
feature building
1.context info of message: author context, conversational context, temperal context
2.lexical info of message: discourse(cue words, question, long) content(repeat)

We used the development set to approximate the normal densities used in our context models and the evaluation set to obtain
the results reported below.
'''
ann1done = pd.read_csv(r'../records/annotation/anno1done.csv')
ann2done = pd.read_csv(r'../records/annotation/anno2done.csv')

ann1 = pd.read_csv(r'../records/annotation/anno1.csv')
ann2 = pd.read_csv(r'../records/annotation/anno2.csv')

ann1 = pd.concat([ann1,ann1done[['thread','appthread']]],axis  = 1)
del ann1done
ann2 = pd.concat([ann2,ann2done[['thread','appthread']]],axis  = 1)
del ann2done

ann1.createTime = pd.to_datetime(ann1.createTime)
ann2.createTime = pd.to_datetime(ann2.createTime)

ann1.dropna(subset = ['msg','thread'],how = 'any', inplace = True)
ann2.dropna(subset = ['msg','thread'],how = 'any', inplace = True)

ann1 = ann1[ann1.thread > -2]
ann2 = ann2[ann2.thread > -2]
#set the time window 群组讨论的当天和前后半天
twindow = 0.5#这个应该用在全局的totalmsg表上，每次处理响应窗口的数据
#%%
#%%
'''
the author context of a message m, denoted by CA(m), is the set of other messages
written by m’s author am:
'''
#context info of message
#author context
    
totalTimediff = []
for row in ann1.itertuples():
    msgid = row[2]
    sender = row[6]
    msgdate = row[4]
    maxdate = msgdate + datetime.timedelta(days = 1)
    mindate = msgdate - datetime.timedelta(days = 1)
    threadid = row[10] 
    autherContext = ann1[(ann1.sender == sender)&(ann1.msgSvrId != msgid)]
    timediff = autherContext[autherContext.thread == threadid]['createTime'] - msgdate
    timediff = timediff.tolist()#单位秒
    timediff = [x.total_seconds()/60 for x in timediff]
    totalTimediff.extend(timediff)

#估计这个timediff的参数
plt.hist(totalTimediff, normed = True)
st.norm.fit(totalTimediff)#loc = 0 scale = 58.7
#%%
#%%
'''
conversational context 
'''
totalTimediff = []
for row in ann1.itertuples():
    msgid = row[2]
    sender = row[6]
    msgdate = row[4]
    maxdate = msgdate + datetime.timedelta(days = 1)
    mindate = msgdate - datetime.timedelta(days = 1)
    threadid = row[10]
    #我提到了那些人
    mentionNames = msgat[(msgat.msgSvrId == msgid)&(msgat.createTime > mindate)&(msgat.createTime < maxdate)]['member_x'].tolist()
    #我被那些人提到
    mentionNames.extend(msgat[(msgat.member_x == sender)&(msgat.createTime > mindate)&(msgat.createTime < maxdate)]['sender'].tolist())
    #这些人说的话
    converContext = ann1[ann1.sender.isin(mentionNames)&(ann1.createTime > mindate)&(ann1.createTime < maxdate)]
    timediff = converContext[converContext.thread == threadid]['createTime'] - msgdate
    timediff = timediff.tolist()
    totalTimediff.extend([x.total_seconds()/60 for x in timediff])
    
#估计这个timediff的参数
plt.hist(totalTimediff, normed = True)
#减去均值，使其为0
avg = sum(totalTimediff) / float(len(totalTimediff))
totalTimediff = [x - avg for x in totalTimediff]
st.norm.fit(totalTimediff)#loc = 0 scale = 9.6
#%%

#%%
'''
temporal context

'''
totalTimediff = []
for row in ann1.itertuples():
    msgid = row[2]
    sender = row[6]
    msgdate = row[4]
    maxdate = msgdate + datetime.timedelta(days = 1)
    mindate = msgdate - datetime.timedelta(days = 1)
    threadid = row[10]
    
    temporalContext = ann1[(ann1.createTime > mindate)&(ann1.createTime < maxdate)&(ann1.msgSvrId != msgid)]
    timediff = temporalContext[temporalContext.thread == threadid]['createTime'] - msgdate
    timediff = timediff.tolist()
    totalTimediff.extend([x.total_seconds()/60 for x in timediff])
    
#估计这个timediff的参数
plt.hist(totalTimediff, normed = True)
#减去均值，使其为0
avg = sum(totalTimediff) / float(len(totalTimediff))
totalTimediff = [x - avg for x in totalTimediff]
st.norm.fit(totalTimediff)#loc = 0 scale = 117
#%%

In [3]:
#%%
'''
compute the context-free freatures.
represent sentences with vectors of terms counts.
Terms:
    1.bag of words
    2.length ?
'''
#get bag of words
#split sentences
'''
#问题：是否对msg做一些过滤，字段
1.通过停止词
2.是否过滤掉@微信名这种
3.过滤掉只出现过一次的词
'''
re_filter = re.compile(u'@(?P<atname>.*?)\u2005')
ann1['msg'] = ann1.msg.str.replace(re_filter,'')#replace the @wechatid with '', this should be done when deal with predict dataset.
ann2['msg'] = ann2.msg.str.replace(re_filter,'')

import os
LTP_DATA_DIR = r'D://ltp_data//ltp_data_v3.4.0'  # ltp模型目录的路径
cws_model_path = os.path.join(LTP_DATA_DIR, 'cws.model')  # 分词模型路径，模型名称为`cws.model`
#在此基础上加入分词的语句
from pyltp import Segmentor
segmentor = Segmentor()  # 初始化实例
segmentor.load(cws_model_path)  # 加载模型

wordsLists = []

for index, msg in ann2['msg'].iteritems():
    #print(msg)
    sents = SentenceSplitter.split(msg)
    #sentsLists.append(sents)
    wordslist = []
    for sent in sents:
        #过滤句子中的标点符号
        words = segmentor.segment(sent)
        wordslist.extend(list(words))
    wordsLists.append(wordslist)
ann2['splitwords'] = wordsLists

segmentor.release()  # 释放模型,分词完毕
'''
#the filted msg may contain nothing, cause everything is filted
#remove common words
with open(r'stopwords.tab', encoding = 'utf-8') as f_stop:
     stopwords = [line.strip() for line in f_stop]
     
ann2['splitwords'] = [[word for word in wordslist if word not in stopwords] for wordslist in ann2['splitwords'].tolist()]
'''
#%%

#%%
'''
BUILD THE BAG OF WORDS
'''
from gensim import corpora, models
from six import iteritems
dictionary = corpora.Dictionary(ann2.splitwords.tolist())
#read stopwords list
with open(r'stopwords.tab', encoding = 'utf-8') as f_stop:
     stopwords = [line.strip() for line in f_stop]
#remove stop words and words appear only once
stop_ids = [dictionary.token2id[stopword] for stopword in stopwords if stopword in dictionary.token2id]
#alternative stop_ids
#dictionary.filter_n_most_frequent(50)
once_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq == 1]

dictionary.filter_tokens(stop_ids + once_ids)
dictionary.compactify()# remove gaps in id sequence after words that were removed

#initialize the bag of words model
corpus = [dictionary.doc2bow(text) for text in ann2['splitwords'].tolist()]
tfidf = models.TfidfModel(corpus)
corpus_tfidf =  tfidf[corpus]
#model persistency
#dictionary.save('ann2_split_words.dict')#store the dictionary for future reference
#corpus_tfidf.save('corpus_tfidf')
'''
for doc in corpus_tfidf:
    print(doc)
'''
ann2['tfidf'] = list(corpus_tfidf)
#convert from list of tuple to list of dicts
ann2['tfidf'] = list(map(lambda x: dict(x), ann2['tfidf']))
#%%

#%%
'''
BY NOW THE TFIDF MODEL IS THE CONTEXT FREE REPRESENTATION OF MESSAGES.
NOW I WILL EXPAND THIS REPRESENTATION WITH CONTEXT INFO WITH PRAMETERS TRAINED BEFORE
HOW TO EXPAND: FOR EACH MESSAGE i BUILD THE FOLLOWING FUNCTIONS
    AUTHOR CONTEXT PROB autherProb(msgi)
    CONVERSATIONAL CONTEXT PROB converProb(msgi)
    TEMPORAL CONTEXT PROB
    BOTH OF THESE FUNCTIONS RESTRICT THE TIMEWINDOW BETWEEN [DATE - 1DAY,DATE + 1DAY](to achieve this, build a df by date)
    
'''
def probMultiTfidf(probArray, tfidfList):
    '''Multiply the context tfidf with probability in the same thread.
    Args:
        probArray(Array): probability in the same thread of msg context
        tfidfList(List): the tfidfList of msg context
    Returns:
        The return tuple
    '''    
    '''
    rList = []
    rappend = rList.append
    for i in range(len(probArray)):
        r2List = []
        r2append = r2List.append
        #at the very beginning, the tfidfList is a list of tuple
        #fixed this to dict
        for term, freq in tfidfList[i].items():
            #r2List.append((term, freq*probArray[i]))
            r2append((term, freq*probArray[i]))
        #rList.append(r2List)
        rappend(r2List)
    return(rList)
    '''
    #alternative
    return([{k: v for k,v in zip(tfidfList[i].keys(),probArray[i] * np.array(list(tfidfList[i].values())))} for i in range(len(probArray))])
def sumDicts(dict1,dict2):
    '''Sum dict1 with dict2, only 2.
    
    '''
    return({k:dict1.get(k,0) + dict2.get(k,0) for k in dict1.keys()|dict2.keys()})    
def sumTfidf(tfidfList):
    '''Sum the new computed tfidf
    Args:
        tfidfList: A list of tfidf
    Return:
        the return list: expanded tfidf dict
    '''
    rList = {}
    for tfidf in tfidfList:
        '''
        for term, freq in tfidf:
            if(term in rList):#has_key was removed in Python3
                rList[term] = rList[term] + freq
            else:
                rList[term] = freq
        '''
        #alternative
        rList = sumDicts(tfidf,rList)
    return(rList)
def autherProb(row, t_scale, contextdf, w_auther):
    '''Expand the context-free info of msgi with auther context.
    
    Args:
        row(Series): One raw of msgdf.itertuples() to be expanded
        t_scale(float): The scale of normal distribution trained before
        contextdf(dataframe): The dataframe of msgs between [DATE - 1DAY,DATE + 1DAY]
        w_auther(float): Weight for auther context.
    Returns:
        The return vector. The expand vector repretation of msgi with info from its auther context.
    '''
    msgid = row[2]
    sender = row[6]
    msgdate = row[4]
    #maxdate = msgdate + datetime.timedelta(days = 1)
    #mindate = msgdate - datetime.timedelta(days = 1)
    #threadid = row[10] 
    #based on the defination, the autherContext dont have to be in the same thread, just the same auther.
    #FIXED
    autherContext = contextdf[(contextdf.sender == sender)&(contextdf.msgSvrId != msgid)]
    timediff = autherContext['createTime'] - msgdate
    timediff = timediff.tolist()#in seconds
    timediff = [x.total_seconds()/60for x in timediff]
    tfidfList = autherContext['tfidf'].tolist()
    probArray = st.norm.pdf(timediff, scale = t_scale)
    newTfidf = probMultiTfidf(probArray,tfidfList)

    #return(sumTfidf(newTfidf))
    return({k:(w_auther*v)  for k,v in sumTfidf(newTfidf).items()})

def converProb(row, t_scale, contextdf, msgat, w_conver):
    '''Expand the context-free info of msgi with conversational context.
    
    Args:
        row(Series): One raw of msgdf.itertuples() to be expanded
        t_scale(float): The scale of normal distribution trained before
        contextdf(dataframe): The dataframe of msgs between [DATE - 1DAY,DATE + 1DAY]
        msgat(dataframe): The dataframe of msgat between [DATE - 1DAY,DATE + 1DAY]
                        , for the reduction of repeted searching the bounded date of msgat
        w_conver(float): weight for conversational context
    Returns:
        The return vector. The expand vector repretation of msgi with info from its conversational context.    
    '''
    msgid = row[2]
    sender = row[6]
    msgdate = row[4]
    #maxdate = msgdate + datetime.timedelta(days = 1)
    #mindate = msgdate - datetime.timedelta(days = 1)
    #threadid = row[10] 
    #autherContext = contextdf[(contextdf.sender == sender)&(contextdf.msgSvrId != msgid)&(contextdf.thread == threadid)]
    #我提到了那些人
    mentionNames = msgat[msgat.msgSvrId == msgid]['member_x'].tolist()
    #我被那些人提到
    mentionNames.extend(msgat[msgat.member_x == sender]['sender'].tolist())
    #这些人说的话
    converContext = contextdf[(contextdf.sender.isin(mentionNames))]
    
    timediff = converContext['createTime'] - msgdate
    timediff = timediff.tolist()#in seconds
    timediff = [x.total_seconds()/60for x in timediff]
    tfidfList = converContext['tfidf'].tolist()
    probArray = st.norm.pdf(timediff, scale = t_scale)
    newTfidf = probMultiTfidf(probArray,tfidfList)

    return({k:(v*w_conver) for k,v in sumTfidf(newTfidf).items()})    
def tempProb(row, t_scale, contextdf, w_temp):
    '''Expand the context-free info of msgi with temporal context.
    
    Args:
        row(Series): One raw of msgdf.itertuples() to be expanded
        t_scale(float): The scale of normal distribution trained before
        contextdf(dataframe): The dataframe of msgs between [DATE - 1DAY,DATE + 1DAY]

    Returns:
        The return vector. The expand vector repretation of msgi with info from its temporal context. 
    
    '''
    msgid = row[2]
    #sender = row[6]
    msgdate = row[4]

    #threadid = row[10] 
    tempContext = contextdf[(contextdf.msgSvrId != msgid)]
    timediff = tempContext['createTime'] - msgdate
    timediff = timediff.tolist()#in seconds
    timediff = [x.total_seconds()/60for x in timediff]
    tfidfList = tempContext['tfidf'].tolist()
    probArray = st.norm.pdf(timediff, scale = t_scale)
    newTfidf = probMultiTfidf(probArray,tfidfList)

    return({k:(v*w_temp)for k,v in sumTfidf(newTfidf).items()})

def expandedMsg(contextFree, autherContext, converContext, tempContext, w_contextFree):
    '''Sum the context-free msg tfidf with all context info.
    Args:
        contextFree(List): The tfidf list of the raw msg, each element is a dict
        autherContext(List): The tfidf List of expanded auther context info.
        tempContext(List): The tfidf List of expanded temporal context info.
        converContext(List): The tfidf List of expanded conversational context info.
        w_contextFree(float): Weights for context-free
    Returns:
        The return vector of expanded representation.
    '''
    rList = []
    for i in range(len(contextFree)):
        #rList.append(sumDicts(sumDicts(sumDicts(contextFree[i],autherContext[i]),converContext[i]),tempContext[i]))
        contextPart ={k:(v*(1-w_contextFree))for k,v in sumDicts(sumDicts(autherContext[i],converContext[i]),tempContext[i]).items()}
        contextFreePart = {k:(v*w_contextFree) for k,v in contextFree[i].items()}
        rList.append(sumDicts(contextFreePart, contextPart))
    return(rList)
auther_scale = 58.7
conver_scale = 9.6
temporal_scale = 117

w_contextFree = 0.45
w_auther = 0.3
w_conver = 0.6
w_temp = 1-w_auther-w_conver
#weights = {'w_contextFree':w_contextFree,'w_context':w_context,'w_auther':w_auther,'w_conver':w_conver,'w_temp':w_temp}
#access the date property with a .dt accessor
autherExpandList = []
converExpandList = []
tempExpandList = []
for date in ann2['createTime'].dt.date.unique():
    #FIEXED
    maxdate = datetime.datetime(date.year,date.month,date.day) + datetime.timedelta(days = twindow+1)
    mindate = datetime.datetime(date.year,date.month,date.day) - datetime.timedelta(days = twindow)
    #slice the msgat dataframe with bounded time period
    msgatdf = msgat[(msgat.createTime > mindate)&(msgat.createTime < maxdate)]
    contextdf = ann2[(ann2.createTime > mindate)&(ann2.createTime < maxdate) ]
    targetdf = ann2[ann2.createTime.dt.date == date] 
    for row in targetdf.itertuples():
        autherExpandList.append(autherProb(row, auther_scale, contextdf, w_auther))
        converExpandList.append(converProb(row, conver_scale, contextdf, msgatdf, w_conver))
        tempExpandList.append(tempProb(row, temporal_scale, contextdf, w_temp))
        #break

ann2['extended'] = expandedMsg(ann2['tfidf'].tolist(),autherExpandList,converExpandList,tempExpandList, w_contextFree)

#ann2.to_csv('../records/annotation/anno2extented.csv')
#%%

#%%
'''Compute the similarity between a given msg and existing threads
Single pass clustering is performed.(reference about single pass tech: 
    http://facweb.cs.depaul.edu/mobasher/classes/csc575/assignments/single-pass.html)
1.Treat the first msg as a single-message cluster T.
2.for each remaining msg m compute for All the existing threads.
    sim(m, T)= max sim(m, mi) mi belongs to All the existing threads.
    sim(m, mi) = cosin similarity between these two msgs.
'''
def dictNorm(dict1):
    '''Compute the norm of a dictionary just like a vector
    Args:
        dict1(dict): the square of sum of each value in this dict.
    Returns:
        The return float.
    '''    
    return(sum([v*v for v in dict1.values()]))
def dotProduct(dict1,dict2):
    '''Compute the dot product of two dicts, like vectors
    Args:
        dict1, dict2(dicts): two tfidf dicts.
    Returns:
        The return float.
    '''
    return(sum([dict1[k]*dict2[k] for k in dict1.keys()&dict2.keys()]))

def similarity(msgdf, targetmsgid, msgdate, threadDict, threshold):
    '''Pairwise similarity function.
    
    1. Turn the date into date counts.
    2. The composation of Theadid:
        Part1: Date count * 10e4
        Part2: increamental thread id counts.(suppose this count will not exceed 10e3)
        Theadid = Part1 + Part2
    3. Only compute in the range of [Date - 1, Date]
    Args:
        msgdf(dataframe): The dataframe of msg log
        targetmsgid(String): The target unique msg id to be compared.
        msgdate(int): The date count from the beginning date.
        threadDict(dict): key= thread id, value = List of msgid.
        threshold(float): if the max similarity is under threshold, start a new cluster containing only this msg
    Returns:
        The updated threadDict.
    '''
    if(len(threadDict) == 0):
        threadid = 1 + 10000*msgdate
        threadDict[threadid] = [targetmsgid]
    else:
        max_similarity_thread = 0
        max_similarity = 0
        #find the max similarity and the corresponding thread over all threads.
        for thread, msgids in threadDict.items():
            #Notice, the threadDict may be NULL!
            threaddate = thread // 10000
            if(threaddate < msgdate - 1): continue
            for msgid in msgids:
                targetmsg = msgdf[msgdf.msgSvrId == targetmsgid]['extended'].item()#get the exact dict rather than dict and object type
                comparedmsg = msgdf[msgdf.msgSvrId == msgid]['extended'].item()
                cosine = dotProduct(targetmsg,comparedmsg)/math.sqrt(dictNorm(targetmsg)*dictNorm(comparedmsg))
                if (cosine > max_similarity):
                    max_similarity = cosine
                    max_similarity_thread = thread
                     
        if(max_similarity > threshold):
            #print(max_similarity)
            threadDict[max_similarity_thread].append(targetmsgid)
        else:
            #create a new thread
            if(threaddate == msgdate):
                threadDict[max(threadDict.keys())%10000 + 1 + msgdate*10000] = [targetmsgid]
            if(threaddate < msgdate):
                #if this a new date and new thread, then reset the thread id to 1.
                threadDict[1 + msgdate*10000] = [targetmsgid]
    return(threadDict)    

D:\Anaconda2\envs\py3\lib\site-packages\matplotlib\axes\_axes.py:6462: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "
D:\Anaconda2\envs\py3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Detect the threads.
AND
Test the time.

In [4]:
#%%time
# cell magic starts with %% and line magic starts with %
threadDict = {}       
begin_date = min(ann2['createTime'])
threshold = 0.7
# Use count to control how many msgs to distengle thread.
count = 1
for row in ann2.itertuples():
    msgid = row[2]
    msgdate = row[4]
    msgdate = (msgdate - begin_date).days + 1
    threadDict = similarity(ann2, msgid, msgdate, threadDict, threshold)
    #should not use multiprocessing, cause the thread detection is in the time sequence.
    
    #count += 1
    #if(count >50): break

#TODO
# compare the performance between for loop map() and list comprehension    
# map is not suitable for this iteration.
# and list comprehension is not suitable too.
#%%

In [19]:
#for k,v in threadDict.items():
#    print(k, len(v))

In [6]:
realThreadDict = {}
for thread in ann2['thread'].unique():
    msgs = ann2[ann2['thread'] == thread]['msgSvrId'].tolist()
    realThreadDict[thread] = msgs

In [7]:
#%%
'''Choose the F value to be the object and try to train the optimal model.
1. Build the F value.
2. Adjust the parameters to maximize F value.
''' 
def recall(ti,tj):
    '''The recall between the real thread i and the detected thread j.
    
    Recall(i,j) = nij / ni
    where nij is the number of msgs of the real thread i in the detected thread j.
    ni is the number of msgs in the real thread i.

    Args:
        ti(int): The real thread number, also the key in the realThreadDict.
        tj(int): The detected thread number, also the key in the threadDict.
    Return:
        The return float number.    
    '''
    realMsg = realThreadDict[ti]
    detectMsg = threadDict[tj]
    nij = len([real for real in realMsg if real in detectMsg])
    ni = len(realMsg)
    #check the result with jupyter console
    #good result
    return(nij/ni)
def precision(ti,tj):
    '''Precision(i,j) = nij / nj
    where nj is the number of msgs in the detected thread j.
    
    Args:
        ti(int): The real thread number, also the key in the realThreadDict.
        tj(int): The detected thread number, also the key in the threadDict.
    Return:
        The return float number.        
    '''
    realMsg = realThreadDict[ti]
    detectMsg = threadDict[tj]
    nij = len([real for real in realMsg if real in detectMsg])
    nj = len(detectMsg)
    #check the result with jupyter console
    #good result
    return(nij/nj)
def pairf(ti,tj):
    ''' F(i,j) = 2*Precision*Recall /(Precision + Recall)
    is the F measure of detected thread j and the real thread i.
    '''  
    prevalue = precision(ti,tj)
    revalue = recall(ti,tj)
    #print(prevalue, revalue)
    if((prevalue ==0 )|( revalue == 0)): 
        return(0)
    else:
        return(2*prevalue*revalue/(prevalue+revalue))

def fvalue(realThreadDict, threadDict):
    '''The whole F measure of the detection result in a stream
is defined as a weighted sum over all threads as follow.
    Args:
        realThreadDict(dict): The dict of ground true threads of msgs. Key is thread id and value is msgid list.
        threadDict(dict): The dict of detected thread.
    '''
    max_pairf, wholef = 0, 0
    len_msg = len(ann2)
    for realThread in realThreadDict:
        for detectThread in threadDict:
            value_pairf = pairf(realThread, detectThread)
            if(value_pairf > max_pairf): max_pairf = value_pairf
        wholef = wholef + len(realThreadDict[realThread])*value_pairf

    return wholef/len_msg

#%%

In [8]:
fvalue(realThreadDict, threadDict)

0.00261519302615193

In [18]:
def tuning(w_contextFree = 0.45,w_auther = 0.3, w_conver = 0.6, w_temp = 0.1, threshold = 0.7):
    '''Tuning parameters.
    1. Compute the extended msg tfidf with all the parameters.
    2. Cluster msgs into threads.
    3. Compute the F value.

    Args:
        w_*, threshold(float): all the weights and threshold to be tuning.

    ''' 
    #access the date property with a .dt accessor
    autherExpandList = []
    converExpandList = []
    tempExpandList = []
    for date in ann2['createTime'].dt.date.unique():
        #FIEXED
        maxdate = datetime.datetime(date.year,date.month,date.day) + datetime.timedelta(days = twindow+1)
        mindate = datetime.datetime(date.year,date.month,date.day) - datetime.timedelta(days = twindow)
        #slice the msgat dataframe with bounded time period
        msgatdf = msgat[(msgat.createTime > mindate)&(msgat.createTime < maxdate)]
        contextdf = ann2[(ann2.createTime > mindate)&(ann2.createTime < maxdate) ]
        targetdf = ann2[ann2.createTime.dt.date == date] 
        for row in targetdf.itertuples():
            autherExpandList.append(autherProb(row, auther_scale, contextdf, w_auther))
            converExpandList.append(converProb(row, conver_scale, contextdf, msgatdf, w_conver))
            tempExpandList.append(tempProb(row, temporal_scale, contextdf, w_temp))
    
    #ann2['tfidf'] = list(map(lambda x: dict(x), ann2['tfidf']))
    #ann2['extended'] = expandedMsg(ann2['tfidf'].tolist(),autherExpandList,converExpandList,tempExpandList, w_contextFree)
    #extendedtfidf is a list of dicts.
    extendedtfidf = expandedMsg(ann2['tfidf'].tolist(),autherExpandList,converExpandList,tempExpandList, w_contextFree)
    
    #Then compute the similarity and get the thread.
    threadDict = {}       
    begin_date = min(ann2['createTime'])
    for row in ann2.itertuples():
        msgid = row[2]
        msgdate = row[4]
        msgdate = (msgdate - begin_date).days + 1
        threadDict = similarity(ann2, msgid, msgdate, threadDict, threshold)
    
    #Then compute the F value.
    return(fvalue(realThreadDict, threadDict))

In [ ]:

#search best parameters
contextFreeList = [i/10 for i in range(1,3,step = 0.5)]
#%debug
for i in contextFreeList:
    print(i,tuning(w_contextFree = i))

In [ ]:
%debug
tuning()

造成contextfree参数变化但是F不变的原因在于 context free的tfidf和 extended 的tfidf相比大了太多（e-1/e-5）
,所以应该直接使用term freq进行扩充吗？
不是，更主要的原因出现在auther_Prob等的概率上。

In [11]:
ann2['tfidf'].iloc[0]

{0: 0.4817966406077289,
 1: 0.222270827298042,
 2: 0.3228899823130633,
 3: 0.3425837264331833,
 4: 0.3589708703355047,
 5: 0.3685892530799719,
 6: 0.4817966406077289}

In [21]:
ann2['extended'].iloc[0]

{0: 0.21691880859509494,
 1: 0.1019265818802901,
 2: 0.14583174092960416,
 3: 0.1576901578720162,
 4: 0.16302535459193637,
 5: 0.16737925162080566,
 6: 0.21717417975746264,
 7: 0.0008171642775682974,
 8: 0.00159157647203461,
 9: 0.002115964978626841,
 10: 0.0002821783650919936,
 11: 0.0002149099542170135,
 12: 0.0035055325968951423,
 13: 0.0017829703883001935,
 14: 0.0020136067722110675,
 15: 0.0016161010311329892,
 16: 0.000252772827285321,
 17: 0.0006418371073635242,
 18: 0.00362947804448041,
 19: 0.0015054808310182974,
 20: 0.00021154123907689152,
 21: 0.0013726418386152551,
 22: 6.382745798036741e-05,
 23: 0.0003616348498169077,
 24: 0.0006404030843569339,
 25: 0.0019271209435752419,
 26: 0.00169940924902967,
 27: 0.0004410466303800156,
 28: 0.001588785861273323,
 29: 0.00028272745443085095,
 30: 0.00010559023683213355,
 31: 0.0002513514442865208,
 32: 8.919141671928322e-05,
 33: 0.0010169269733844638,
 34: 0.00027481035959706795,
 35: 0.00040290201657068977,
 36: 0.002053664002560